In [62]:
import pandas as pd

import numpy as np
import pandas as pd

sets = pd.read_csv('lego/sets.csv')
inventory_parts = pd.read_csv('lego/inventory_parts.csv')
inventories = pd.read_csv('lego/inventories.csv')
parts = pd.read_csv('lego/parts.csv')
print(parts)
biggest_set = sets.sort_values('num_parts', ascending=False).iloc[0:10]
print(biggest_set)


      part_num                                               name  part_cat_id
0       0687b1                        Set 0687 Activity Booklet 1           17
1         0901  Baseplate 16 x 30 with Set 080 Yellow House Print            1
2         0902  Baseplate 16 x 24 with Set 080 Small White Hou...            1
3         0903     Baseplate 16 x 24 with Set 080 Red House Print            1
4         0904  Baseplate 16 x 24 with Set 080 Large White Hou...            1
...        ...                                                ...          ...
25988   zbb013              Znap Connector 3 x 3 - 4 way B (Beam)           43
25989   zbb014                     Znap Connector 1 x 3 - 2 way A           43
25990   zbb015                                Znap Beam 3, 1 Hole           43
25991   zbb018            Znap Connector 3 x 3 - 4 way C (Closed)           43
25992   zbb022                  Wheel 68mm Znap Propeller (9 x 2)           29

[25993 rows x 3 columns]
         set_num          

In [63]:
total_quantity = inventory_parts.loc[inventory_parts['inventory_id'] == 76, 'quantity'].sum()
print(total_quantity)

5962


In [64]:
list1=[]

for index,row_series in biggest_set.iterrows():
    list1.append({'set_num': row_series['set_num'], 'set_name': row_series['name']})
print(list1)

[{'set_num': '10189-1', 'set_name': 'Taj Mahal'}, {'set_num': 'SWMP-1', 'set_name': 'Star Wars / M&M Mosaic - Promo Set'}, {'set_num': '2000409-1', 'set_name': 'Window Exploration Bag'}, {'set_num': '10179-1', 'set_name': 'Millennium Falcon - UCS'}, {'set_num': '75827-1', 'set_name': 'Firehouse Headquarters'}, {'set_num': '40179-1', 'set_name': 'Personalised Mosaic Portrait'}, {'set_num': '10214-1', 'set_name': 'Tower Bridge'}, {'set_num': '10253-1', 'set_name': 'Big Ben'}, {'set_num': '71040-1', 'set_name': 'The Disney Castle'}, {'set_num': '75159-1', 'set_name': 'Death Star'}]


In [65]:
import pandas as pd

result_list = []

# Assuming list1 contains set information (set_num, set_name)
for set_info in list1:
    set_num = set_info['set_num']
    set_name = set_info['set_name']

    # Find the corresponding inventory information
    inventory_info = inventories[inventories['set_num'] == set_num].iloc[0]

    # Extract inventory ID
    inventory_id = inventory_info['id']

    # Filter inventory parts based on inventory ID
    set_inventory_parts = inventory_parts[inventory_parts['inventory_id'] == inventory_id]

    # Append a dictionary with set, inventory, and parts information to the result list
    result_list.append({
        'set_num': set_num,
        'set_name': set_name,
        'inventory_id': inventory_id,
        'parts_data': set_inventory_parts[['part_num', 'quantity']].to_dict(orient='records')
    })

# Convert the list of dictionaries to a DataFrame
result_df = pd.DataFrame(result_list)

# Explode the 'parts_data' column to separate rows for each part_num and quantity
result_df = result_df.explode('parts_data')

# Extract 'part_num' and 'quantity' from the 'parts_data' dictionary column
result_df[['part_num', 'quantity']] = pd.DataFrame(result_df['parts_data'].tolist(), index=result_df.index)

# Merge with the 'parts' DataFrame to get 'part_name'
result_df = pd.merge(result_df, parts[['part_num', 'name']], on='part_num', how='left')

# Drop the original 'parts_data' column
result_df = result_df.drop('parts_data', axis=1)

# Display the final result DataFrame
print(result_df)




      set_num    set_name  inventory_id part_num  quantity  \
0     10189-1   Taj Mahal            76     2339         8   
1     10189-1   Taj Mahal            76     2357        12   
2     10189-1   Taj Mahal            76    2412b        60   
3     10189-1   Taj Mahal            76     2419        12   
4     10189-1   Taj Mahal            76     2420        36   
...       ...         ...           ...      ...       ...   
2986  75159-1  Death Star          7869    98138         1   
2987  75159-1  Death Star          7869    98286         1   
2988  75159-1  Death Star          7869    98313         4   
2989  75159-1  Death Star          7869    99780         2   
2990  75159-1  Death Star          7869    99781         4   

                                              name  
0            Brick Arch 1 x 5 x 4 [Continuous Bow]  
1                               Brick 2 x 2 Corner  
2     Tile Special 1 x 2 Grille with Bottom Groove  
3                    Wedge Plate 3 x 6 Cut 

In [66]:
part_num_counts = result_df.groupby('part_num').filter(lambda x:len(x)> 1)

# Display the part numbers that are in multiple sets
duplicate_part_nums = part_num_counts['part_num'].unique()
print("Part numbers present in multiple sets:")
print(duplicate_part_nums)

Part numbers present in multiple sets:
['2339' '2357' '2412b' '2419' '2420' '2431' '2436' '2450' '2453a' '2454a'
 '2456' '2730' '2780' '2877' '3001' '3002' '3003' '3004' '30044' '3005'
 '3006' '3008' '3009' '3010' '3020' '3021' '3022' '3023' '3024' '3030'
 '3031' '3032' '3033' '3034' '3035' '30363' '30367b' '30374' '30414'
 '30503' '30505' '3062b' '3068b' '3069b' '3070b' '32018' '32028' '32324'
 '3245b' '32531' '32532' '32555' '3307' '3460' '3622' '3623' '3626b'
 '3659' '3660' '3665' '3666' '3680' '3700' '3701' '3702' '3707' '3710'
 '3737' '3794a' '3795' '3832' '3857' '3941' '3958' '4032a' '4070' '4081b'
 '4085c' '4162' '43337' '43898' '44375a' '44567a' '44568' '44728' '4477'
 '4490' '4510' '4589' '4740' '47457' '47905' '4864b' '4865a' '52107'
 '54200' '59443' '6019' '60471' '60474' '60479' '6111' '6112' '6141'
 '6180' '6182' '6231' '6232' '6259' '6558' '6587' '6636' '73983' '4186'
 '2458' '3039' '3040b' '3065' '3957b' '4175' '4286' '4287' '4495b'
 '970c00' '2432' '2445' '2462' '2465' 

In [59]:
filename = 'lego_result.csv'
result_df.to_csv(filename, index=False)
print(f'DataFrame has been saved as CSV file: "{filename}"')

DataFrame has been saved as CSV file: "lego_result.csv"


In [69]:
df = pd.DataFrame(result_df)

# Create a new DataFrame with unique part numbers and their associated names
unique_parts_df = df[['part_num', 'name']].drop_duplicates()

# Display the resulting DataFrame
print(unique_parts_df)
filename = 'unique.csv'
unique_parts_df.to_csv(filename, index=False)
print(f'DataFrame has been saved as CSV file: "{filename}"')

          part_num                                          name
0             2339         Brick Arch 1 x 5 x 4 [Continuous Bow]
1             2357                            Brick 2 x 2 Corner
2            2412b  Tile Special 1 x 2 Grille with Bottom Groove
3             2419                 Wedge Plate 3 x 6 Cut Corners
4             2420                            Plate 2 x 2 Corner
...            ...                                           ...
2981  973pr3523c01                     MINI UPPER PART, NO. 3523
2982  973pr3524c01                     MINI UPPER PART, NO. 3524
2983  973pr3533c01                     MINI UPPER PART, NO. 3533
2984         98108          Minifig Helmet SW Death Star Trooper
2988         98313                Arm Mechanical [Thick Support]

[826 rows x 2 columns]
DataFrame has been saved as CSV file: "unique.csv"


In [70]:
unique_df = df.drop_duplicates(subset=['inventory_id', 'set_name'])

# Display the resulting DataFrame
print(unique_df)

        set_num                            set_name  inventory_id   part_num  \
0       10189-1                           Taj Mahal            76       2339   
174      SWMP-1  Star Wars / M&M Mosaic - Promo Set         12161       3020   
204   2000409-1              Window Exploration Bag          7847       2458   
238     10179-1             Millennium Falcon - UCS          6409  10179cert   
536     75827-1              Firehouse Headquarters          8461      10201   
1223    40179-1        Personalised Mosaic Portrait         18438       3024   
1230    10214-1                        Tower Bridge         11520      12825   
1437    10253-1                             Big Ben         11169      10928   
1700    71040-1                   The Disney Castle         12397      10183   
2400    75159-1                          Death Star          7869      10201   

      quantity                                               name  
0            8              Brick Arch 1 x 5 x 4 [C

,set_num,set_name,inventory_id,part_num,quantity
0,10189-1,Taj Mahal,76,2339,8
0,10189-1,Taj Mahal,76,2357,12
0,10189-1,Taj Mahal,76,2412b,60
0,10189-1,Taj Mahal,76,2419,12
0,10189-1,Taj Mahal,76,2420,36
...,...,...,...,...,...
0,10189-1,Taj Mahal,76,3623,4
0,10189-1,Taj Mahal,76,3626b,6
0,10189-1,Taj Mahal,76,3626b,16
0,10189-1,Taj Mahal,76,3659,2
